In [ ]:
# 1. Configurações e preparações iniciais.

# Bibliotecas importadas
import pandas as pd
import os
import openpyxl
from pandas.io.formats import excel
from glob import glob

# Estilização final para exportação do arquivo no formato padrão do Excel.
pd.options.mode.chained_assignment = None
excel.ExcelFormatter.header_style = None

# IMPORTANTE!!! -- A SOLUÇÃO IMPLEMENTADA PRECISA QUE A PASTA EM QUE SE ENCONTRAM OS ARQUIVOS .xlsx NÃO TENHA NADA ALÉM DOS ARQUIVOS EM QUESTÃO!
# NADA ALÉM DOS ARQUIVOS mês.xlsx DENTRO DA PASTA FONTE/DESTINO!!

# Exemplo: D:\Desafio_ISOps\Pastas_com_Arquivos_Excel
CAMINHO_PARA_OS_ARQUIVOS_A_SEREM_AGREGADOS = ''
ARQUIVOS_FILTRADOS = glob(os.path.join(CAMINHO_PARA_OS_ARQUIVOS_A_SEREM_AGREGADOS, '*.xlsx'))
# Filtra a busca apenas pelos arquivos com formato .xlsx.

# Dataframe/Tabela que será utilizado para agregação.
dados_consolidados = pd.DataFrame()
dados_consolidados.copy = False

# Lista temporária para realizar a agregação.
dados_temp = []

In [ ]:
# 2. Agregação dos dados de todos os arquivos XLSX dentro do caminho especificado em um unico arquivo.

for ARQUIVO in ARQUIVOS_FILTRADOS:
    dados = pd.read_excel(ARQUIVO, 'Página1')
    dados_temp.append(dados)
dados_consolidados = pd.concat(dados_temp, ignore_index=True)

In [ ]:
# 3. Verificação dos tipos de dados de das células.

'''
A função verificar_tipo_de_dado_da_celula é usada para verificar se o tipo de dado de dada célula de uma coluna específica
está no formato datetime64[ns] (formato padrão do Excel para células do tipo Data).

Possíveis resultados:
Verdadeiro: O valor correto para atender ao critério de formatação ISO8601 é atribuido à célula.
Falso: Um valor nulo (NA) é atribuido à célula.
'''

def verificar_tipo_de_dado_da_celula(valor_da_celula):
    if pd.api.types.is_datetime64_any_dtype(valor_da_celula):
        return pd.NA if pd.isna(valor_da_celula) else valor_da_celula.date()
    else:
        if valor_da_celula == 0:
            return pd.NA
        else:
            return valor_da_celula

for coluna in dados_consolidados.columns[0:]:
    dados_consolidados[coluna] = dados_consolidados[coluna].apply(verificar_tipo_de_dado_da_celula)

In [ ]:
# 4. Exportar a tabela final para um arquivo no formato .xlsx(Excel).

dados_consolidados.to_excel('Dados Agregados.xlsx', sheet_name='Página1', index=False)